In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.3.0


In [14]:
# checkpoint function
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [3]:
# data load
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis = -1)
    test_data = np.expand_dims(test_data, axis = -1)
    
    train_data, test_Data = normalize(train_data, test_data)
    
    train_labels = to_categorical(train_labels, 10)
    test_labels = to_categorical(test_labels, 10)
    
    return train_data, train_labels, test_data, test_labels

# pre_processing
def normalize(train_data, test_data) :
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0
    
    return train_data, test_data

In [4]:
# cost function
def loss_fn(model, images, labels):
    logits = model(images, training = True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels,from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [5]:
def flatten():
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias = True, kernel_initializer = weight_init)

def relu():
    return tf.keras.layers.Activation(tf.keras.activations.relu)

In [9]:
# create model(function version)
def create_model_function(label_dim):
    weight_init = tf.keras.initializers.RandomNormal()
    
    model = tf.keras.Sequential()
    model.add(flatten())
    
    for i in range(2):
        model.add(dense(256, weight_init))
        model.add(relu())
        
    model.add(dense(label_dim, weight_init))
    
    return model

In [15]:
# dataset
train_x, train_y, test_x, test_y = load_mnist()

# parameters
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) 

label_dim = 10

train_flag = True

# Graph Input using Dataset API
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [16]:
# Model
network = create_model_function(label_dim)

# Training 
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

model_dir = 'nn_relu'

# writer
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [17]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):                
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)
                
                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/60000] time: 0.9375, train_loss: 2.16200852, train_accuracy: 0.2969, test_Accuracy: 0.1681
Epoch: [ 0] [    1/60000] time: 1.2397, train_loss: 2.29011822, train_accuracy: 0.1562, test_Accuracy: 0.2488
Epoch: [ 0] [    2/60000] time: 1.5429, train_loss: 2.29734325, train_accuracy: 0.1094, test_Accuracy: 0.3341
Epoch: [ 0] [    3/60000] time: 1.7623, train_loss: 2.28512359, train_accuracy: 0.1719, test_Accuracy: 0.3871
Epoch: [ 0] [    4/60000] time: 1.9448, train_loss: 2.28626728, train_accuracy: 0.2266, test_Accuracy: 0.4137
Epoch: [ 0] [    5/60000] time: 2.1303, train_loss: 2.29033208, train_accuracy: 0.1797, test_Accuracy: 0.4272
Epoch: [ 0] [    6/60000] time: 2.3248, train_loss: 2.28832722, train_accuracy: 0.1406, test_Accuracy: 0.4411
Epoch: [ 0] [    7/60000] time: 2.5133, train_loss: 2.29333401, train_accuracy: 0.1562, test_Accuracy: 0.4533
Epoch: [ 0] [    8/60000] time: 2.7177,

Epoch: [ 0] [   73/60000] time: 17.2529, train_loss: 2.09068346, train_accuracy: 0.3281, test_Accuracy: 0.7663
Epoch: [ 0] [   74/60000] time: 17.4423, train_loss: 2.02598619, train_accuracy: 0.3438, test_Accuracy: 0.7720
Epoch: [ 0] [   75/60000] time: 17.6378, train_loss: 2.04761934, train_accuracy: 0.3047, test_Accuracy: 0.7811
Epoch: [ 0] [   76/60000] time: 17.8473, train_loss: 2.00165534, train_accuracy: 0.3359, test_Accuracy: 0.7864
Epoch: [ 0] [   77/60000] time: 18.0308, train_loss: 2.00848699, train_accuracy: 0.3906, test_Accuracy: 0.7881
Epoch: [ 0] [   78/60000] time: 18.2163, train_loss: 1.93317068, train_accuracy: 0.3594, test_Accuracy: 0.7916
Epoch: [ 0] [   79/60000] time: 18.3978, train_loss: 1.98880196, train_accuracy: 0.2656, test_Accuracy: 0.7901
Epoch: [ 0] [   80/60000] time: 18.6312, train_loss: 2.06740642, train_accuracy: 0.3906, test_Accuracy: 0.7928
Epoch: [ 0] [   81/60000] time: 18.8994, train_loss: 1.99096036, train_accuracy: 0.3672, test_Accuracy: 0.7927
E

Epoch: [ 0] [  147/60000] time: 34.3362, train_loss: 1.70528769, train_accuracy: 0.3906, test_Accuracy: 0.7525
Epoch: [ 0] [  148/60000] time: 34.6134, train_loss: 1.83547950, train_accuracy: 0.4219, test_Accuracy: 0.7533
Epoch: [ 0] [  149/60000] time: 34.8588, train_loss: 1.73665404, train_accuracy: 0.4141, test_Accuracy: 0.7635
Epoch: [ 0] [  150/60000] time: 35.0542, train_loss: 1.78527880, train_accuracy: 0.3828, test_Accuracy: 0.7756
Epoch: [ 0] [  151/60000] time: 35.2308, train_loss: 1.79754436, train_accuracy: 0.4062, test_Accuracy: 0.7843
Epoch: [ 0] [  152/60000] time: 35.4223, train_loss: 1.74915159, train_accuracy: 0.3672, test_Accuracy: 0.7923
Epoch: [ 0] [  153/60000] time: 35.6068, train_loss: 1.78263021, train_accuracy: 0.3125, test_Accuracy: 0.7975
Epoch: [ 0] [  154/60000] time: 35.7893, train_loss: 1.73492265, train_accuracy: 0.3750, test_Accuracy: 0.8027
Epoch: [ 0] [  155/60000] time: 35.9708, train_loss: 1.92195261, train_accuracy: 0.3281, test_Accuracy: 0.8053
E

Epoch: [ 0] [  221/60000] time: 49.8247, train_loss: 1.59518421, train_accuracy: 0.4219, test_Accuracy: 0.8253
Epoch: [ 0] [  222/60000] time: 50.0023, train_loss: 1.80112553, train_accuracy: 0.3438, test_Accuracy: 0.8282
Epoch: [ 0] [  223/60000] time: 50.1947, train_loss: 1.70522904, train_accuracy: 0.3984, test_Accuracy: 0.8306
Epoch: [ 0] [  224/60000] time: 50.3862, train_loss: 1.71419477, train_accuracy: 0.3984, test_Accuracy: 0.8365
Epoch: [ 0] [  225/60000] time: 50.5697, train_loss: 1.56783986, train_accuracy: 0.4375, test_Accuracy: 0.8399
Epoch: [ 0] [  226/60000] time: 50.7722, train_loss: 1.65745413, train_accuracy: 0.4141, test_Accuracy: 0.8449
Epoch: [ 0] [  227/60000] time: 50.9757, train_loss: 1.63220954, train_accuracy: 0.4375, test_Accuracy: 0.8495
Epoch: [ 0] [  228/60000] time: 51.1711, train_loss: 1.71232057, train_accuracy: 0.4453, test_Accuracy: 0.8506
Epoch: [ 0] [  229/60000] time: 51.3636, train_loss: 1.72810602, train_accuracy: 0.3984, test_Accuracy: 0.8527
E

Epoch: [ 0] [  294/60000] time: 65.2126, train_loss: 1.62257004, train_accuracy: 0.4219, test_Accuracy: 0.8554
Epoch: [ 0] [  295/60000] time: 65.4001, train_loss: 1.64369774, train_accuracy: 0.4688, test_Accuracy: 0.8588
Epoch: [ 0] [  296/60000] time: 65.5926, train_loss: 1.63063860, train_accuracy: 0.4375, test_Accuracy: 0.8568
Epoch: [ 0] [  297/60000] time: 65.7741, train_loss: 1.71883798, train_accuracy: 0.3906, test_Accuracy: 0.8544
Epoch: [ 0] [  298/60000] time: 65.9596, train_loss: 1.50777340, train_accuracy: 0.4531, test_Accuracy: 0.8529
Epoch: [ 0] [  299/60000] time: 66.1700, train_loss: 1.78825951, train_accuracy: 0.3594, test_Accuracy: 0.8494
Epoch: [ 0] [  300/60000] time: 66.3934, train_loss: 1.61406446, train_accuracy: 0.4219, test_Accuracy: 0.8493
Epoch: [ 0] [  301/60000] time: 66.5779, train_loss: 1.63615632, train_accuracy: 0.4219, test_Accuracy: 0.8497
Epoch: [ 0] [  302/60000] time: 66.7824, train_loss: 1.62932336, train_accuracy: 0.4609, test_Accuracy: 0.8519
E

Epoch: [ 0] [  368/60000] time: 81.5070, train_loss: 1.68912053, train_accuracy: 0.4219, test_Accuracy: 0.8600
Epoch: [ 0] [  369/60000] time: 81.8072, train_loss: 1.59921026, train_accuracy: 0.4609, test_Accuracy: 0.8603
Epoch: [ 0] [  370/60000] time: 82.1373, train_loss: 1.73472416, train_accuracy: 0.3359, test_Accuracy: 0.8599
Epoch: [ 0] [  371/60000] time: 82.3916, train_loss: 1.66398573, train_accuracy: 0.4453, test_Accuracy: 0.8607
Epoch: [ 0] [  372/60000] time: 82.5851, train_loss: 1.71244717, train_accuracy: 0.3594, test_Accuracy: 0.8604
Epoch: [ 0] [  373/60000] time: 82.7746, train_loss: 1.67407751, train_accuracy: 0.4219, test_Accuracy: 0.8606
Epoch: [ 0] [  374/60000] time: 82.9581, train_loss: 1.68010736, train_accuracy: 0.3828, test_Accuracy: 0.8597
Epoch: [ 0] [  375/60000] time: 83.1506, train_loss: 1.62192893, train_accuracy: 0.4688, test_Accuracy: 0.8606
Epoch: [ 0] [  376/60000] time: 83.3381, train_loss: 1.84088278, train_accuracy: 0.2734, test_Accuracy: 0.8608
E

Epoch: [ 0] [  442/60000] time: 97.4823, train_loss: 1.55021453, train_accuracy: 0.4688, test_Accuracy: 0.8741
Epoch: [ 0] [  443/60000] time: 97.6827, train_loss: 1.56801975, train_accuracy: 0.4766, test_Accuracy: 0.8747
Epoch: [ 0] [  444/60000] time: 97.8662, train_loss: 1.55293763, train_accuracy: 0.5078, test_Accuracy: 0.8756
Epoch: [ 0] [  445/60000] time: 98.0597, train_loss: 1.64420557, train_accuracy: 0.3750, test_Accuracy: 0.8778
Epoch: [ 0] [  446/60000] time: 98.2542, train_loss: 1.69915235, train_accuracy: 0.3438, test_Accuracy: 0.8779
Epoch: [ 0] [  447/60000] time: 98.4567, train_loss: 1.65788579, train_accuracy: 0.4062, test_Accuracy: 0.8766
Epoch: [ 0] [  448/60000] time: 98.6461, train_loss: 1.59219718, train_accuracy: 0.3906, test_Accuracy: 0.8761
Epoch: [ 0] [  449/60000] time: 98.8546, train_loss: 1.53805041, train_accuracy: 0.4453, test_Accuracy: 0.8756
Epoch: [ 0] [  450/60000] time: 99.0441, train_loss: 1.56766248, train_accuracy: 0.4844, test_Accuracy: 0.8751
E